In [ ]:
# --- Required Libraries ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

In [ ]:
# --- Physical Constants ---
h = 6.62607015e-34  # Planck constant [J·s]
e = 1.602176634e-19  # Elementary charge [C]
hbar = h / (2 * np.pi)

In [ ]:
# --- EC and EJ Values in GHz ---
EC_ghz = np.array([0.0470, 0.0197, 0.0294, 0.0088, 0.4494, 0.1799, 0.1772])
EJ_ghz = np.array([13.622] * 7)  # Same EJ for all qubits

In [ ]:
# --- Convert GHz to Joules ---
def ghz_to_joules(freq_ghz):
    return freq_ghz * 1e9 * h

EC = ghz_to_joules(EC_ghz)
EJ = ghz_to_joules(EJ_ghz)

In [ ]:
# --- Calculate ω01 (GHz) ---
omega_01_ghz = (np.sqrt(8 * EJ * EC) - EC) / h * 1e-9  # GHz

In [ ]:
# --- Simulate T1 and T2 in QuTiP ---
def simulate_t1_t2(decay_rate, dephasing_rate, t_max=20e-6):
    a = destroy(2)
    tlist = np.linspace(0, t_max, 1000)

    # T1: Excited state decay
    psi1 = basis(2, 1)
    c_ops_t1 = [np.sqrt(decay_rate) * a]
    result_t1 = mesolve(0 * a, psi1, tlist, c_ops_t1, [a.dag() * a])

    # T2: Superposition decoherence
    psi2 = (basis(2, 0) + basis(2, 1)).unit()
    c_ops_t2 = [np.sqrt(decay_rate) * a, np.sqrt(dephasing_rate) * a.dag() * a]
    result_t2 = mesolve(0 * a, psi2, tlist, c_ops_t2, [a.dag() * a])

    return tlist, result_t1.expect[0], result_t2.expect[0]

In [ ]:
# --- Use Sample Relaxation/Dephasing Times ---
T1 = 20e-6  # 20 µs
T2 = 30e-6  # 30 µs
decay_rate = 1 / T1
dephasing_rate = 1 / T2 - decay_rate / 2

In [ ]:
# --- Run Simulation ---
tlist, pop_t1, pop_t2 = simulate_t1_t2(decay_rate, dephasing_rate)

In [ ]:
# --- Estimate Resonator Frequencies (Δ = -1 GHz) ---
omega_r_ghz = omega_01_ghz + 1.0

In [ ]:
# --- Plot Results ---
plt.figure(figsize=(16, 10))

# ω01 Frequencies
plt.subplot(2, 2, 1)
plt.bar(range(7), omega_01_ghz, tick_label=[f"Q{i}" for i in range(7)], color='dodgerblue')
plt.ylabel("ω01 (GHz)")
plt.title("Qubit Frequencies")

# T1 Plot
plt.subplot(2, 2, 2)
plt.plot(tlist * 1e6, pop_t1, label="T1 decay", color='crimson')
plt.xlabel("Time (µs)")
plt.ylabel("Excited State Population")
plt.title("T1 Relaxation")

# T2 Plot
plt.subplot(2, 2, 3)
plt.plot(tlist * 1e6, pop_t2, label="T2 decay", color='seagreen')
plt.xlabel("Time (µs)")
plt.ylabel("Coherence")
plt.title("T2 Decoherence")

# Resonator Frequencies
plt.subplot(2, 2, 4)
plt.bar(range(7), omega_r_ghz, tick_label=[f"Q{i}" for i in range(7)], color='orange')
plt.ylabel("ωr (GHz)")
plt.title("Readout Resonator Frequencies")

plt.tight_layout()
plt.show()